In [141]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer

##### 1-3) Загрузим датасет, считаем  в Python. Подготовим для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [5]:
data = np.genfromtxt('SMSSpamCollection.txt', dtype=None, delimiter='\t')

In [108]:
text = [( i[1].decode('utf8')) for i in data]
target = [(0 if i[0].decode('utf8') == 'ham' else 1) for i in data]

4) Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получим из списка текстов матрицу признаков X

In [122]:
cvr = CountVectorizer()
X = cvr.fit_transform(texts)

5) Оценим качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Параметр cv зададим равным 10. В качестве метрики качества используем f1-меру класса 1 (то есть, без микро и макро усреднения). Получившееся качество – ответ в этом пункте.

In [123]:
logreg = LogisticRegression(random_state=1)
scores = cross_val_score(logreg, X, target, scoring='f1', cv=10)
print('Mean score:', scores.mean())

Mean score: 0.933419572279


6) Обучим классификатор на всей выборке и спрогнозируем с его помощью класс для следующих сообщений:

In [124]:
text1 = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?", 
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
    "Only 99$"
]
logreg.fit(X, target)
result = logreg.predict(cvr.transform(text1))
print(result)

[1 1 0 0 0]


7) n-граммы

In [127]:
logreg_ngram_scores = []
for ng_range in [(1,1),(2,2), (3,3), (1,3)]:
    vect = CountVectorizer(ngram_range=ng_range)
    X = vect.fit_transform(text)
    logreg_ngram_scores.append(cross_val_score(logreg, X, target, scoring='f1', cv=10).mean())

print([i for i in logreg_ngram_scores])

[0.93341957227863492, 0.81857220331211278, 0.72396573537867059, 0.92362874602119605]


8) тоже самое для  MultinomialNB()

In [129]:
MNB = MultinomialNB()

MNB_ngram_scores = []
for ng_range in [(1,1),(2,2), (3,3), (1,3)]:
    vect = CountVectorizer(ngram_range=ng_range)
    X = vect.fit_transform(text)
    MNB_ngram_scores.append(cross_val_score(MNB, X, target, scoring='f1', cv=10).mean())


print([i for i in MNB_ngram_scores])


[0.92767639962595572, 0.64493971901126523, 0.38059278664201324, 0.89071678099186435]


9) Tfidfvectorizer на униграммах

In [139]:
tfidfv = TfidfVectorizer(ngram_range=(1,1))
X = tfidfv.fit_transform(text)
print('Mean score:', cross_val_score(logreg, X, target, scoring='f1', cv=10).mean())

Mean score: 0.852662726124


ниже, чем с CountVectorizer() на униграммах(0.85 < 0.93)

10) Попробуем получить как можно более высокое качество на кросс-валидации(используем токенизацию, стэмминг и удаление стоп-слов)

сначала Tfidf

In [147]:
tokenizer = TweetTokenizer(reduce_len=True)
X = []
for i in range(len(text)):
    X.append(((tokenizer.tokenize(text[i]))))
    for j in range(len(X[i])):
        X[i][j] = PorterStemmer().stem(X[i][j])
    X[i] = " ".join(X[i])
tfidf = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
X = tfidf.fit_transform(X)
# prepare a range of alpha values to test
alphas = np.array( [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000])
# create and fit a ridge regression model, testing each alpha
model = LogisticRegression()
# model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(C=alphas), cv=5)
grid.fit(X, target)
print(grid)
# summarize the results of the grid search
print('\n')
print('Best score =', grid.best_score_)
print('\n')
print('значение коэффициента С', grid.best_estimator_.C)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   5.00000e-01,   1.00000e+00,
         5.00000e+00,   1.00000e+01,   1.00000e+02,   2.00000e+02,
         5.00000e+02,   1.00000e+03,   1.00000e+04,   1.50000e+04,
         2.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)


Best score = 0.980086114101


значение коэффициента С 100000.0


теперь CountVectorizer

In [143]:
X = []
for i in range(len(text)):
    X.append(((tokenizer.tokenize(text[i]))))
    for j in range(len(X[i])):
        X[i][j] = PorterStemmer().stem(X[i][j])
    X[i] = " ".join(X[i])
vect = CountVectorizer(ngram_range=(1,1), stop_words='english')
X = vect.fit_transform(text)
# prepare a range of alpha values to test
alphas = np.array( [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000])
# create and fit a ridge regression model, testing each alpha
model = LogisticRegression()
# model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(C=alphas), cv=5)
grid.fit(X, target)
print(grid)
# summarize the results of the grid search
print('\n')
print('Best score =', grid.best_score_)
print('\n')
print('значение коэффициента С', grid.best_estimator_.C)
vect = CountVectorizer(ngram_range=ng_range)
X = vect.fit_transform(text)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   5.00000e-01,   1.00000e+00,
         5.00000e+00,   1.00000e+01,   1.00000e+02,   2.00000e+02,
         5.00000e+02,   1.00000e+03,   1.00000e+04,   1.50000e+04,
         2.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)


Best score = 0.983494797273


значение коэффициента С 20000.0


### Вывод:

1) Для задачи бинарной классификации текстов SMS-сообщений CountVectorizer лучше подходит в качестве способа отбора признаков, нежели TfidfVectorizer.

2) При выборе других моделей(не лог-регрессии) результаты ухудшались

3) Предобработка текста идёт на пользу качеству классификации)

4)  n-граммы не помогли